Load model and data.

In [1]:
import theano
import theano.tensor as T
import numpy as np
import cPickle
import random
from utils import *
from state import *
from discri_model import DiscriModel

theano.config.floatX='float32'

model_name = 'model/train_discri_full_emb256_h128x128_layer2_model.npz'

state = prototype_state()
model = DiscriModel(state)
model.load(model_name)

print('Model loaded.')

(word2ind, ind2word) = cPickle.load(open('tmp/dict.pkl'))
acttype = cPickle.load(open('tmp/acttype_cnt.pkl'))
print acttype.keys()

print('Data loaded.')

Model loaded.
[u'requestaddr', u'informpricerange', u'requestarea', u'negate', u'requestpricerange', u'reqmore', u'informname', u'denyfood', u'confirmpricerange', u'requestsignature', u'requestphone', u'informarea', u'reqalts', u'informfood', u'denyname', u'repeat', u'confirmarea', u'requestfood', u'informthis', u'requestname', u'restart', u'affirm', u'bye', u'thankyou', u'ack', u'confirmfood', u'hello', u'requestpostcode']
Data loaded.


We randomly sample some data from the dev set and see the results.

In [18]:
def process(test_sent):
    global acttype, ind2word, word2ind
    print "Input sent:", test_sent
    words = test_sent.split()
    nat_coded = [1]
    for w in words:
        if w in word2ind:
            nat_coded.append(word2ind[w])
        else:
            print '  out of vocab: %s' % w
    nat_coded.append(0)
    # print 'Coded input:', nat_coded

    m = state['seqlen']
    nat_coded_mat = numpy.zeros((m, 1), dtype='int32')
    sent_len = len(nat_coded)
    nat_coded_mat[:sent_len, 0] = nat_coded
    pred_fn = theano.function([model.natural_input],
                                    model.pred)
    pred = pred_fn(nat_coded_mat)

    # print 'Prediction (coded):', pred

    def decode_act(acttype, pred):
        res = {}
        pos = 0
        for act in acttype.keys():
            res[act] = []
            k = max(1, acttype[act])
            while k > 0:
                res[act].append(pred[pos])
                pos = pos + 1
                k = k - 1
        return res

    
    res = decode_act(acttype, pred)
    for act in res.keys():
        if ind2word[res[act][0]] != '<NO>':
            print "%s: %s" % (act, ' '.join([ind2word[ind] for ind in res[act]]))
    print
    
dev_data = cPickle.load(open('tmp/dev_data.pkl'))
random.shuffle(dev_data)

for k in range(10):
    test_sent = dev_data[k][1]
    process(test_sent)

Input sent: is there any serving vietnamese
reqalts: <YES>
informfood: vietnamese <NULL>

Input sent: thare are different expensive restaurants in the east part of town
informpricerange: expensive
informarea: east

Input sent: postal code
requestpostcode: <YES>

Input sent: that serves restaurant in south part of town
informarea: south

Input sent: im looking for a restaurant in the west part of town
informarea: west

Input sent: can i have the address and the price range
requestaddr: <YES>
requestpricerange: <YES>

Input sent: address
requestaddr: <YES>

Input sent: west
informarea: west

Input sent: steakhouse
informfood: steakhouse <NULL>

Input sent: how about indian food
reqalts: <YES>
informfood: indian <NULL>



Now we try to mannually provide an input sentence (if a word is out of vocab, we simply remove it).

In [19]:
test_set = ['thank you', \
            'goodbye', \
            'i want to have some expensive chinese food', \
            'can i have some australian food at town center', 'is there any other choice', \
            'i need a cheap restaurant serving italian food', \
            'i need a cheap restaurant serving chinese food', \
            'whats the location', \
            'how about the price', \
            'i would like something cheap', \
            'tell me a british restaurant', \
            # let's try something abnormal
            'tell me a british restaurant and give me the phone number', \
            'tell me a british restaurant and give me the phone number and the address', \
            'tell me a british restaurant and give me the phone number and the address thank you and good bye']


            
for sent in test_set:
    process(sent)

Input sent: thank you
thankyou: <YES>

Input sent: goodbye
bye: <YES>

Input sent: i want to have some expensive chinese food
informfood: mediterranean <NULL>

Input sent: can i have some australian food at town center
informarea: centre
informfood: french <NULL>

Input sent: is there any other choice
reqalts: <YES>
informfood: cantonese <NULL>

Input sent: i need a cheap restaurant serving italian food
informpricerange: cheap
informfood: italian <NULL>

Input sent: i need a cheap restaurant serving chinese food
informpricerange: cheap
informfood: brazilian <NULL>

Input sent: whats the location
requestarea: <YES>

Input sent: how about the price
informpricerange: dontcare

Input sent: i would like something cheap
informpricerange: cheap

Input sent: tell me a british restaurant
informfood: vietnamese <NULL>

Input sent: tell me a british restaurant and give me the phone number
reqalts: <YES>
informfood: chinese <NULL>

Input sent: tell me a british restaurant and give me the phone num